In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from enum import Enum
import time
import pyautogui
# !python -m pip install mediapipe opencv-python matplotlib pyautogui
# !conda deactivate

In [2]:
HTTP = 'https://'
IP_ADDRESS = '192.168.1.10'
URL =  HTTP + IP_ADDRESS + ':4343/video'

cap = cv2.VideoCapture(URL)

In [3]:

horeizontal_threshold = 7
verticall_threshold =1 
class directions(Enum):
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4
    STRAIGHT = 0
def neared_distance(point0,point1,point2,frame):
    x_0 = int(point0.x * frame.shape[1])
    y_0 = int(point0.y * frame.shape[0])
    
    x_1 = int(point1.x * frame.shape[1])
    y_1 = int(point1.y * frame.shape[0])
    
    x_2 = int(point2.x * frame.shape[1])
    y_2 = int(point2.y * frame.shape[0])
    
    # calc the distance between point 0 and point 1
    distance_1 = ((x_0 - x_1)**2 + (y_0 - y_1)**2)**0.5
    
    # calc the distance between point 0 and point 2
    distance_2 = ((x_0 - x_2)**2 + (y_0 - y_2)**2)**0.5
    # print(distance_1,distance_2)
    return distance_1,distance_2

def vertical_decision(mid_point, top_most, bottom_most, frame):
    distance_1,distance_2 = neared_distance(mid_point, top_most, bottom_most,frame)
    print('vertical_destances: ',distance_1,distance_2)
    if abs(distance_1 - distance_2) < verticall_threshold:
        return directions.STRAIGHT
    elif distance_1 < distance_2:
        return directions.UP
    else:
        return directions.DOWN
    
def horizontal_desecion(mid_point, left_most, right_most,frame):
    distance_1,distance_2 = neared_distance(mid_point, left_most, right_most,frame)
    distance_2-=5
    print('horizontal_distances: ',distance_1,distance_2)
    if abs(distance_1 - distance_2) < horeizontal_threshold:
        return directions.STRAIGHT
    elif distance_1 < distance_2:
        return directions.LEFT
    else:
        return directions.RIGHT

def move_mouse(direction):
    if(direction == directions.STRAIGHT):
        return
    current_x, current_y = pyautogui.position()
    if direction == directions.LEFT:
        current_x -= 5
    elif direction == directions.RIGHT:
        current_x += 5
    elif direction == directions.UP:
        current_y += 5
    elif direction == directions.DOWN:
        current_y -= 5
    else:
        print('mouse is in the same position')
    pyautogui.moveTo(int(current_x), int(current_y))

mp_face_mesh = mp.solutions.face_mesh

left_eye_mid_index= 468
left_eye_leftmost = 33
left_eye_rightmost = 246

right_eye_mid_index = 473
right_eye_leftmost = 362
right_eye_rightmost = 466

left_eye_boundary = [33, 133, 173, 157, 158, 159, 160, 161, 246, 163, 144, 145, 153, 154, 155, 133]
right_eye_boundary = [362, 263, 373, 380, 381, 382, 384, 385, 386, 387, 388, 466, 390, 373, 374, 380]

In [5]:

mp_face_mesh = mp.solutions.face_mesh

with mp_face_mesh.FaceMesh(max_num_faces=1,
                           refine_landmarks=True,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Ignoring empty camera frame.")
            break
        result = face_mesh.process(frame)
        for face_landmarks in result.multi_face_landmarks:
            right_mid_point = face_landmarks.landmark[right_eye_mid_index]
            right_rightmost_point = face_landmarks.landmark[right_eye_rightmost]
            right_leftmost_point = face_landmarks.landmark[right_eye_leftmost]
            horeizontal_dirction = horizontal_desecion(right_mid_point, right_rightmost_point, right_leftmost_point, frame)
            pt1=right_mid_point
            x = int(pt1.x * frame.shape[1])
            y = int(pt1.y * frame.shape[0])
            cv2.circle(frame, (x,y), 2, (0,255,0), -1)
            
            pt1=right_rightmost_point
            x = int(pt1.x * frame.shape[1])
            y = int(pt1.y * frame.shape[0])
            cv2.circle(frame, (x,y), 2, (0,255,0), -1)
            
            pt1=right_leftmost_point
            x = int(pt1.x * frame.shape[1])
            y = int(pt1.y * frame.shape[0])
            cv2.circle(frame, (x,y), 2, (0,255,0), -1)            
            # print(horeizontal_dirction)
            
            
            left_mid_point = face_landmarks.landmark[left_eye_mid_index]
            left_rightmost_point = face_landmarks.landmark[left_eye_rightmost]
            left_leftmost_point = face_landmarks.landmark[left_eye_leftmost]
            # vertical_direction = horizontal_desecion(left_mid_point, left_rightmost_point, left_leftmost_point, frame)
            
            pt1=left_rightmost_point
            x = int(pt1.x * frame.shape[1])
            y = int(pt1.y * frame.shape[0])
            cv2.circle(frame, (x,y), 2, (0,255,0), -1)
            
            pt1=left_leftmost_point
            x = int(pt1.x * frame.shape[1])
            y = int(pt1.y * frame.shape[0])
            cv2.circle(frame, (x,y), 2, (0,255,0), -1)     
            # print(vertical_direction)

            for i in range(0,468):
                pt1 = face_landmarks.landmark[i]
                x = int(pt1.x * frame.shape[1])
                y = int(pt1.y * frame.shape[0])
                # cv2.circle(frame, (x,y), 2, (0,255,0), -1)
        cv2.imshow('frame', cv2.flip(frame,1))
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        # time.sleep(2)

    
cap.release()
cv2.destroyAllWindows()